# CLTV Prediction With BG-NBD and Gamma-Gamma

In [1]:
pip install lifetimes

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pytz>=2017.2 in c:\users\guldeniz\anaconda3\lib\site-packages (from pandas>=0.24.0->lifetimes) (2020.1)



### Libraries

In [2]:
import datetime as dt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter

pd.set_option('display.max_columns', None)

pd.set_option('display.max_columns', None)
df_ = pd.read_excel(r"E:\4. HAFTA\online_retail_II.xlsx",
                    sheet_name="Year 2010-2011")
df = df_.copy()
df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Data Preperation

In [4]:
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(3))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

    
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


def crm_data_prep(dataframe):
    dataframe.dropna(axis=0, inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    return dataframe


In [5]:
check_df(df)

##################### Shape #####################
(406830, 8)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
##################### Head #####################
  Invoice StockCode                         Description  Quantity  \
0  536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6   
1  536365     71053                 WHITE METAL LANTERN         6   
2  536365    84406B      CREAM CUPID HEARTS COAT HANGER         8   

          InvoiceDate  Price  Customer ID         Country  
0 2010-12-01 08:26:00   2.55      17850.0  United Kingdom  
1 2010-12-01 08:26:00   3.39      17850.0  United Kingdom  
2 2010-12-01 08:26:00   2.75      17850.0  United Kingdom  
##################### NA #####################
Invoice        0
Stoc

In [6]:
df_prep = crm_data_prep(df)
check_df(df_prep)

##################### Shape #####################
(397925, 9)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
TotalPrice            float64
dtype: object
##################### Head #####################
  Invoice StockCode                         Description  Quantity  \
0  536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER       6.0   
1  536365     71053                 WHITE METAL LANTERN       6.0   
2  536365    84406B      CREAM CUPID HEARTS COAT HANGER       8.0   

          InvoiceDate  Price  Customer ID         Country  TotalPrice  
0 2010-12-01 08:26:00   2.55      17850.0  United Kingdom       15.30  
1 2010-12-01 08:26:00   3.39      17850.0  United Kingdom       20.34  
2 2010-12-01 08:26:00   2.75      17850.0  United Kingdom     

### Creating RFM Segments

In [7]:
def create_rfm(dataframe):
    

    today_date = dt.datetime(2011, 12, 11)

    rfm = dataframe.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                                'Invoice': lambda num: num.nunique(),
                                                "TotalPrice": lambda price: price.sum()})

    rfm.columns = ['recency', 'frequency', "monetary"]

    rfm = rfm[(rfm['monetary'] > 0)]


    rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])


    rfm['rfm_segment'] = rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)

    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

    rfm['rfm_segment'] = rfm['rfm_segment'].replace(seg_map, regex=True)
    rfm = rfm[["recency", "frequency", "monetary", "rfm_segment"]]
    return rfm


rfm = create_rfm(df_prep)
rfm.head()


,recency,frequency,monetary,rfm_segment
Customer ID,,,,
12346.0,326,1,310.44,hibernating
12347.0,3,7,4310.00,champions
12348.0,76,4,1770.78,at_risk
12349.0,19,1,1491.72,promising
12350.0,311,1,331.46,hibernating


### Calculated CLTV

In [8]:
def create_cltv_c(dataframe):
    # avg_order_value
    dataframe['avg_order_value'] = dataframe['monetary'] / dataframe['frequency']

    # purchase_frequency
    dataframe["purchase_frequency"] = dataframe['frequency'] / dataframe.shape[0]

    # repeat rate & churn rate
    repeat_rate = dataframe[dataframe.frequency > 1].shape[0] / dataframe.shape[0]
    churn_rate = 1 - repeat_rate

    # profit_margin
    dataframe['profit_margin'] = dataframe['monetary'] * 0.05

    # Customer Value
    dataframe['cv'] = (dataframe['avg_order_value'] * dataframe["purchase_frequency"])

    # Customer Lifetime Value
    dataframe['cltv'] = (dataframe['cv'] / churn_rate) * dataframe['profit_margin']

    # minmaxscaler
    scaler = MinMaxScaler(feature_range=(1, 100))
    scaler.fit(dataframe[["cltv"]])
    dataframe["cltv_c"] = scaler.transform(dataframe[["cltv"]])

    dataframe["cltv_c_segment"] = pd.qcut(dataframe["cltv_c"], 3, labels=["C", "B", "A"])

    dataframe = dataframe[["recency", "frequency", "monetary", "rfm_segment",
                           "cltv_c", "cltv_c_segment"]]

    return dataframe



In [9]:
check_df(rfm)

##################### Shape #####################
(4338, 4)
##################### Types #####################
recency          int64
frequency        int64
monetary       float64
rfm_segment     object
dtype: object
##################### Head #####################
             recency  frequency  monetary  rfm_segment
Customer ID                                           
12346.0          326          1    310.44  hibernating
12347.0            3          7   4310.00    champions
12348.0           76          4   1770.78      at_risk
##################### NA #####################
recency        0
frequency      0
monetary       0
rfm_segment    0
dtype: int64
##################### Quantiles #####################
           0.00      0.05   0.50       0.95        0.99        1.00
recency    1.00    3.0000   51.0   312.0000    369.6300     374.000
frequency  1.00    1.0000    2.0    13.0000     30.0000     210.000
monetary   3.75  110.9455  663.1  5634.3065  16950.4071  266163.525


In [10]:
rfm_cltv = create_cltv_c(rfm)
check_df(rfm_cltv)

##################### Shape #####################
(4338, 6)
##################### Types #####################
recency              int64
frequency            int64
monetary           float64
rfm_segment         object
cltv_c             float64
cltv_c_segment    category
dtype: object
##################### Head #####################
             recency  frequency  monetary  rfm_segment    cltv_c  \
Customer ID                                                        
12346.0          326          1    310.44  hibernating  1.000135   
12347.0            3          7   4310.00    champions  1.025959   
12348.0           76          4   1770.78      at_risk  1.004382   

            cltv_c_segment  
Customer ID                 
12346.0                  C  
12347.0                  A  
12348.0                  A  
##################### NA #####################
recency           0
frequency         0
monetary          0
rfm_segment       0
cltv_c            0
cltv_c_segment    0
dtype: int64

In [11]:
rfm_cltv.head()

,recency,frequency,monetary,rfm_segment,cltv_c,cltv_c_segment
Customer ID,,,,,,
12346.0,326,1,310.44,hibernating,1.000135,C
12347.0,3,7,4310.00,champions,1.025959,A
12348.0,76,4,1770.78,at_risk,1.004382,A
12349.0,19,1,1491.72,promising,1.003110,A
12350.0,311,1,331.46,hibernating,1.000154,C


### Predicted CLTV

In [12]:
def create_cltv_p(dataframe):
    today_date = dt.datetime(2011, 12, 11)

    ## recency kullanıcıya özel dinamik.
    rfm = dataframe.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max()-date.min()).days,
                                                                lambda date: (today_date - date.min()).days],
                                                'Invoice': lambda num: num.nunique(),
                                                'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

    rfm.columns = rfm.columns.droplevel(0)

    rfm.columns = ['recency_cltv_p', 'T', 'frequency', 'monetary']

    rfm["monetary"] = rfm["monetary"] / rfm["frequency"]

    rfm.rename(columns={"monetary": "monetary_avg"}, inplace=True)


    rfm["recency_weekly_cltv_p"] = rfm["recency_cltv_p"] / 7
    rfm["T_weekly"] = rfm["T"] / 7



    rfm = rfm[rfm["monetary_avg"] > 0]

    rfm = rfm[(rfm['frequency'] > 1)]

    rfm["frequency"] = rfm["frequency"].astype(int)

    # BGNBD
    bgf = BetaGeoFitter(penalizer_coef=0.01)
    bgf.fit(rfm['frequency'],
            rfm['recency_weekly_cltv_p'],
            rfm['T_weekly'])

    # exp_sales_1_month
    rfm["exp_sales_1_month"] = bgf.predict(4,
                                           rfm['frequency'],
                                           rfm['recency_weekly_cltv_p'],
                                           rfm['T_weekly'])
    # exp_sales_3_month
    rfm["exp_sales_3_month"] = bgf.predict(12,
                                           rfm['frequency'],
                                           rfm['recency_weekly_cltv_p'],
                                           rfm['T_weekly'])

    # expected_average_profit
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(rfm['frequency'], rfm['monetary_avg'])
    rfm["expected_average_profit"] = ggf.conditional_expected_average_profit(rfm['frequency'],
                                                                             rfm['monetary_avg'])
    #  6 months cltv_p
    cltv = ggf.customer_lifetime_value(bgf,
                                       rfm['frequency'],
                                       rfm['recency_weekly_cltv_p'],
                                       rfm['T_weekly'],
                                       rfm['monetary_avg'],
                                       time=6,
                                       freq="W",
                                       discount_rate=0.01)

    rfm["cltv_p"] = cltv

    # minmaxscaler
    scaler = MinMaxScaler(feature_range=(1, 100))
    scaler.fit(rfm[["cltv_p"]])
    rfm["cltv_p"] = scaler.transform(rfm[["cltv_p"]])

    # rfm.fillna(0, inplace=True)

    # cltv_p_segment
    rfm["cltv_p_segment"] = pd.qcut(rfm["cltv_p"], 3, labels=["C", "B", "A"])

    ## recency_cltv_p, recency_weekly_cltv_p
    rfm = rfm[["recency_cltv_p", "T", "monetary_avg", "recency_weekly_cltv_p", "T_weekly",
               "exp_sales_1_month", "exp_sales_3_month", "expected_average_profit",
               "cltv_p", "cltv_p_segment"]]


    return rfm


rfm_cltv_p = create_cltv_p(df_prep)
check_df(rfm_cltv_p)

##################### Shape #####################
(2845, 10)
##################### Types #####################
recency_cltv_p                int64
T                             int64
monetary_avg                float64
recency_weekly_cltv_p       float64
T_weekly                    float64
exp_sales_1_month           float64
exp_sales_3_month           float64
expected_average_profit     float64
cltv_p                      float64
cltv_p_segment             category
dtype: object
##################### Head #####################
             recency_cltv_p    T  monetary_avg  recency_weekly_cltv_p  \
Customer ID                                                             
12347.0                 365  368    615.714286              52.142857   
12348.0                 282  359    442.695000              40.285714   
12352.0                 260  297    219.542500              37.142857   

              T_weekly  exp_sales_1_month  exp_sales_3_month  \
Customer ID                         

In [13]:
crm_final = rfm_cltv.merge(rfm_cltv_p, on="Customer ID", how="left")
check_df(crm_final)

##################### Shape #####################
(4338, 16)
##################### Types #####################
recency                       int64
frequency                     int64
monetary                    float64
rfm_segment                  object
cltv_c                      float64
cltv_c_segment             category
recency_cltv_p              float64
T                           float64
monetary_avg                float64
recency_weekly_cltv_p       float64
T_weekly                    float64
exp_sales_1_month           float64
exp_sales_3_month           float64
expected_average_profit     float64
cltv_p                      float64
cltv_p_segment             category
dtype: object
##################### Head #####################
             recency  frequency  monetary  rfm_segment    cltv_c  \
Customer ID                                                        
12346.0          326          1    310.44  hibernating  1.000135   
12347.0            3          7   4310.00    c

In [14]:
crm_final.head(10)

,recency,frequency,monetary,rfm_segment,cltv_c,cltv_c_segment,recency_cltv_p,T,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p,cltv_p_segment
Customer ID,,,,,,,,,,,,,,,,
12346.0,326,1,310.44,hibernating,1.000135,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12347.0,3,7,4310.00,champions,1.025959,A,365.0,368.0,615.714286,52.142857,52.571429,0.561684,1.678069,631.911974,2.933517,A
12348.0,76,4,1770.78,at_risk,1.004382,A,282.0,359.0,442.695000,40.285714,51.285714,0.364322,1.087951,463.745539,1.919391,B
12349.0,19,1,1491.72,promising,1.003110,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12350.0,311,1,331.46,hibernating,1.000154,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12352.0,37,8,1756.34,loyal_customers,1.004311,A,260.0,297.0,219.542500,37.142857,42.428571,0.739160,2.206857,224.886669,1.904146,B
12353.0,205,1,89.00,hibernating,1.000011,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12354.0,233,1,1079.40,hibernating,1.001628,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12355.0,215,1,459.40,hibernating,1.000295,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
